In [7]:
# Imports go here
import pandas as pd
import numpy as np
import numpy_financial as npf
from datetime import date
from copy import deepcopy
from IPython.display import display, HTML, display_html
from highcharts import Highchart
from chart_builder import plot_chart
# Pandas Warnings & Settings
pd.options.mode.chained_assignment = None  # default='warn'
pd.options.display.float_format = '{:,.2f}'.format
# Import Unit Loan from the unit_loan notebook
import import_ipynb
from unit_loan import unit_loan

In [8]:
class unit_client():
    def __init__(self):
        # ------------------------------------------------------------
        # CLIENT BASIC INFORMATION
        # ------------------------------------------------------------
        self.client_type = 'Persona XYZ'
        self.client_score = '80'
        self.products = []  # This is where a list of unit_loans will be stored
                            # Storage dict should follow the format below
                            # {
                            # 'product': unit_loan_object,
                            # 'start month': 0, 
                            # 'lifetime_cycles': 1,  <How many times client will renew this product>
                            # 'renewal_rate': 0.40,          <40% of balance is renewed>
                            # 'improvement_at_renewal': 0.8, <Default curve is 80% of previous> 
                            # }
                            
    def validate(self):
        # products can't be empty
        if self.products == []:
            raise Exception("Product List cannot be empty")
        required = ["product", 
                    "start_month", 
                    "renewal_rate", 
                    "improvement_at_renewal"]
        for product in self.products:
            if not all (k in product for k in required):
                raise Exception(f"Product is incomplete - required: {required}")

    def ltv(self):
        self.validate()
        # Create an empty dict to store information about the products
        stats = {}
        # Create Empty DF
        df_ltv = pd.DataFrame(columns=['month', 'cashflow', 'balance'])
        for prod in self.products:
            product = deepcopy(prod['product'])
            # Create an empty dict for this product (saved by unique ID)
            stats[product.uuid] = {
                'start_month_list': [],
                'end_month_list': [],
                'ticket_size_list': [],
                'performing_list': [],
                'fpd30_ever30_list': [],   #stored as a list of tuples
            }
            
            start_month = prod['start_month']
            end_month = start_month + product.term
            # Check how many times this client will renew this loan
            try:
                repeat = prod['lifetime_cycles']
            except KeyError:
                repeat = 1
            # Repeat this product for as many cycles as requested             
            for cycle in range(0, repeat):
                df_p = product.loan_cycle()
                df_p.index =  range(start_month, end_month + 1)
                # Merge this into the LTV dataframe
                df_ltv = df_ltv.add(df_p[['cashflow', 'balance']], fill_value=0)
                # --------------- Repeat Loan -----------------            
                # Calculate second loan from this client
                # Renewal size is renewal_rate x the final "perform and pay" probability
                # Assumption is that clients that pre-pay, do not renew                 
                
                # Store values first
                stats[product.uuid]['start_month_list'].append(start_month)
                stats[product.uuid]['end_month_list'].append(end_month)
                stats[product.uuid]['ticket_size_list'].append(product.ticket_size)
                stats[product.uuid]['fpd30_ever30_list'].append(
                    (product.fpd30, product.ever30))

                # Adjust Months              
                start_month = end_month + 1
                end_month = start_month + product.term
                
                
                # Adjust Loan Notional (Tkt Size)
                performing_clients_at_end = df_p['p_perform_and_pmt'].iloc[-1]
                # Store it
                stats[product.uuid]['performing_list'].append(performing_clients_at_end)
                product.ticket_size = (product.ticket_size *
                                       performing_clients_at_end *
                                       prod['renewal_rate'])
                # Adjust for better credit quality on these clients
                product.fpd30 = product.fpd30 * prod['improvement_at_renewal']
                product.ever30 = product.ever30 * prod['improvement_at_renewal']
                
        df_ltv['month'] = df_ltv.index
        return(df_ltv, stats)
    
    def bar_chart(self, column='cashflow'):
        # Cash Flow Chart
        df, _ = self.ltv()
        x = df['month'].to_list()
        y = df[column].to_list()
        return (plot_chart(
            x, 
            y, 
            'months', 
            column.lower(),
            'bar', 
            column.upper(), 
            f'{column.upper()} x Time'
            )
            )

            
            
            

## Sample Client and Analysis

Let's start by creating a sample client as an empty object and changing some of the assumptions.
For the example below we are assuming the client has 2 products.


In [9]:
# Create empty instance and input assumptions
client = unit_client()

# This info will be used for channel consolidation later
client.client_type = 'Geru A'
client.client_score = 80

# _________________________________________________________________
#
# First product for this client - Unsecured Personal Loan
# _________________________________________________________________
loan = unit_loan()
loan.product_name = 'Unsecured Personal Loan'
loan.rate = 0.04
loan.term = 36
loan.ticket_size = 10000
loan.fpd30 = 0.02
loan.ever30 = 0.30
loan.prepay_start = 0.01
loan.prepay_end = 0.01
loan.refi_start = 0.02
loan.refi_end = 0.01
loan.pd_table.append((15, 0.28))
loan.pd_method = {'method': 'pchip', 'order': 3}

# Attach this loan to the client object
product = {
    'product': loan,
    'start_month': 0,
    'renewal_rate': 0.40,
    'improvement_at_renewal': 0.80
    }
client.products.append(product)

# _________________________________________________________________
#
# Second product for this client - Unsecured Personal Loan 'LIMITINHO'
# _________________________________________________________________
loan = unit_loan()
loan.product_name = 'Limitinho'
loan.rate = 0.08
loan.term = 6
loan.ticket_size = 1000
loan.fpd30 = 0.01
loan.ever30 = 0.05
loan.prepay_start = 0.1
loan.prepay_end = 0.1
loan.refi_start = 0.01
loan.refi_end = 0.01
loan.pd_table.append((3, 0.05))
loan.pd_method = {'method': 'pchip', 'order': 3}

# Attach this loan to the client object
product = {
    'product': loan,
    'start_month': 3,
    'renewal_rate': 0.80,
    'improvement_at_renewal': 0.80,
    'lifetime_cycles': 10,
    }
client.products.append(product)

client.validate()

#### Charts 

In [10]:
client.bar_chart()

In [11]:
client.bar_chart('balance')

#### Copy the whole dataframe to be used in Excel

In [12]:
# ----------------------
#  Copy to Excel 
# ----------------------
ltv, stats = client.ltv()
ltv.to_clipboard(excel=True,sep='\t')

### Create Lifetime Charts of products

In [13]:
# Create A series of product charts to display changes along time
df, stats = client.ltv()
# Creates a series of charts displaying how the products change over time
for product in client.products:
    x = stats[product['product'].uuid]['start_month_list']
    y = stats[product['product'].uuid]['ticket_size_list']
    display(
        plot_chart(x, y, 'Months', 'Ticket Size', 'bar', 
        product['product'].product_name, 
        f"Product | {product['product'].product_name}")
        )
